In [1]:
import pandas as pd
from sklearn import feature_extraction
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot
import seaborn as sns
import random
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Cleaning and merging csv files

-------
# Drop duplicates

## dropna()

In [ ]:
df_dont_touch = pd.read_csv('perfume_merge.csv', index_col = 0)

In [ ]:
df_dont_touch.shape

In [ ]:
df = df_dont_touch.drop_duplicates()

In [ ]:
df.shape

## Reset index

In [ ]:
df.index = range(0,df.shape[0])

In [ ]:
list(df.index==range(0,df.shape[0])).count(True)

## Find and merge duplicate names

In [ ]:
# get all rows that have the same name
df[df.duplicated('name', keep=False)].groupby('designer')['name'].count()

In [ ]:
# identify one of the two copies to delete
to_delete_1 = [2094, 2638, 2738, 2741, 3409, 9852, 37758, 40523, 43831, 1371, 
               1557, 1637, 1418, 1419, 2659, 32101, 1438, 1412, 1410, 1413, 
               1414, 1415, 1564, 1461, 1416, 1631, 1666, 1482, 1506, 1417, 5327, 
               1610, 1373, 1504, 1638, 1421, 29539, 1554, 1639, 1439, 1374, 1375, 
               1376, 1464, 1465, 1578, 1577, 1405, 1480, 1481, 1406, 1407, 1408, 
               1625, 1626, 1627, 1628, 1629, 1409, 1411, 1466, 1467, 1468, 32179, 
               1474, 1475, 1476, 1477, 1478, 1633, 1479, 1463, 1519, 1377, 22278,
               1584, 1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1636, 
               1594, 1595, 1576, 1575, 1425, 1522, 3231, 1523, 1551, 1640, 3234, 
               3238, 20072, 1378, 1667, 1420, 1641, 3244, 11379, 3391, 1665, 1434, 
               1527, 1404, 1569, 1570, 1571, 1573, 1574, 1579, 1580, 1581, 1582, 
               1583, 1483, 1484, 1485, 1486, 1487, 1488, 1490, 1491, 1492, 1493, 
               1494, 1495, 1496, 1497, 1634, 1517, 1661, 1556, 1402, 1518, 1662, 
               42167, 1432, 1663, 1403, 1547, 3383, 3386, 2098, 1664, 1489, 1498, 
               1499, 1602, 1441, 3250, 1502, 1503, 3253, 1642, 1643, 3255, 1379, 
               35948, 1596, 1544, 1635, 1545, 1542, 1548, 1549, 1550, 1457, 1658, 
               3369, 1526, 1458, 1401, 1669, 1660, 1597, 1598, 1381, 1380, 1520, 
               1382, 14542, 1443, 1383, 1444, 1668, 3268, 1505, 1612, 1385, 1533, 
               1454, 1655, 1656, 1430, 1657, 1455, 1632, 1456, 15041, 1400, 1540, 
               1539, 1541, 1543, 1386, 3272, 1387, 1500, 1501, 1521, 17266, 1389, 
               1388, 1390, 1536, 1614, 1613, 1426, 22799, 1651, 1652, 1399, 1653, 
               3352, 1565, 1566, 1567, 4572, 1568, 35628, 3357, 3360, 1525, 1534,
               3278, 3286, 3284, 20103, 3288, 1223, 1524, 4626, 1552, 1558, 1507, 
               1508, 1509, 1510, 1559, 4644, 3338, 3336, 1396, 1649, 1619, 1620, 
               1621, 1622, 1560, 1623, 1428, 1555, 1650, 1608, 1391, 1645, 1427, 
               1392, 1515, 1514, 1615, 1395, 1447, 1646, 1394, 1393, 1511, 1512, 
               1553, 3317, 4649, 1599, 1600, 1601, 14450, 1562, 1537, 1618, 3330, 
               3332, 1648, 1397, 1563, 1398, 1647, 1449, 1450, 1616, 1451, 3299, 
               3300, 1452, 1617, 1561, 5749, 3313, 24831, 4628, 6991
              ]

In [ ]:
df[df.duplicated('name', keep=False)].sort_values(by='name')

In [ ]:
# actually delete one of the two copies
df = df.drop(to_delete_1)

In [ ]:
# check what duplicated entries are left
df[df.duplicated('name', keep=False)].groupby('designer')['name'].count()

In [ ]:
df.shape

In [ ]:
len(df.name.unique())

## Find and drop empty accords

In [ ]:
# find empty accords
df[df.duplicated('accords', keep=False)].sort_values(by='accords')

In [ ]:
# drop 'em
df = df.drop(df[df.accords == '[]'].index)

In [ ]:
df.shape

## Find and drop empty notes

In [ ]:
# find empty notes
df[df.duplicated('notes', keep=False)].sort_values(by='notes')

In [ ]:
# drop those
df = df.drop(df[df.notes == 'NaN'].index)

In [ ]:
df.shape

## Reset index again

In [ ]:
df.index = range(0,df.shape[0])

In [ ]:
df.shape

In [ ]:
list(df.index==range(0,df.shape[0])).count(True)

## Fix weird perfume names

In [ ]:
def fix_names(name):
    """
    input: name from df.name
    returns: cleaned df.name
    """
    name = name.replace('`', "'")
    name = name.replace('’', "'")
    name = name.replace('&amp;', ' and ')
    name = name.replace('…', '...')
    name = name.replace('№', 'No.')
    name = name.replace('[', "")
    name = name.replace(']', "")   
    name = name.replace('®', "")
    name = name.replace('–', '-')
    name = name.replace('±', '')
    name = name.replace('´', "'")
    name = name.replace('♥', '')
    name = name.replace('™', '')
    name = name.replace('‘', "'")
    name = name.replace('|', "")
    name = name.replace('△', "")
    name = name.replace('▽', "")
    name = name.replace('○', "")
    name = name.replace('$', "")
    name = name.replace('&gt;', ">")
    name = name.replace('&lt;', "<")
    name = name.replace('@', "at")
    name = name.replace('D/', "d'")
    name = name.replace('/L ', "")
    name = name.replace('mat;', "Mat")
    name = name.replace('é', "e")
    name = name.replace('à', "a")
    name = name.replace('É', "E")
    name = name.replace('è', "e")
    name = name.replace('  ', " ")
    name = name.replace('"', "")
    name = name.replace('ì', "i")    
    name = name.replace(' + > ', " ")    
    name = name.replace('<3', "")   
    name = name.replace('k̠h̠', "kh") 
    name = name.replace('​', ' ')
    
    return name
df['name'] = df.name.apply(fix_names)

In [ ]:
# for the ones that weren't cleaned by above definition
df.loc[20772, 'name'] = 'Base Man Linn Young for men'
df.loc[20773, 'name'] = 'Club Man Linn Young for men'
df.loc[20774, 'name'] = 'Steel Masculino Linn Young for men'
df = df.drop([41538])

In [ ]:
# check what names are weird ones thta need to be cleaned
k=0
for j in df.name.tolist():
    if not all(i.isalnum() or i.isspace() or i in "^¢=+;/?*#,%°:'.-()!" for i in j):
        k+=1
        print (j)

## Reset index again

In [ ]:
df.index = range(0,df.shape[0])

In [ ]:
df.shape

In [ ]:
list(df.index==range(0,df.shape[0])).count(True)

-------
# Get document term matrices of notes and accords

In [ ]:
# Convert a collection of text documents to a matrix of token counts:
word2vec_sklearn = feature_extraction.text.CountVectorizer()

In [ ]:
# Get sparse matrix of notes
notes_df = pd.DataFrame(word2vec_sklearn.fit_transform(df.notes).toarray(), 
                        columns = word2vec_sklearn.get_feature_names(), 
                        index = df.index)

In [ ]:
# Get sparse matrix of accords
accords_df = pd.DataFrame(word2vec_sklearn.fit_transform(df.accords).toarray(), 
                          columns=word2vec_sklearn.get_feature_names(), 
                          index = df.index).ix[:,120:]

In [ ]:
df[df.reminds != '[]'].shape

In [ ]:
df.shape

In [1]:
def clean_reminds(remind):
    """
    input: df.remind, string
    returns: get the image link from reminds column and put into clean_reminds
    """
    if remind != '[]':
        return re.findall(r'https.*?jpg', remind)

df['clean_reminds'] = df.reminds.apply(clean_reminds)

NameError: name 'df' is not defined

# Get cosine similarity for notes and accords

In [ ]:
cos_notes = cosine_similarity(notes_df)

In [ ]:
cos_accords = cosine_similarity(accords_df)

In [ ]:
cos_accords.shape

In [ ]:
cos_notes.shape

## save cosine similarity matrices

In [ ]:
np.save('cos_accords.npy', cos_accords)
np.save('cos_notes.npy', cos_notes)

# Load files
Save cleaned dataframe

In [ ]:
df.to_csv('perfume_cleaned.csv')

Load cleaned dataframe

In [2]:
df = pd.read_csv('perfume_cleaned.csv', index_col=[0])

Load cos similarity matrix for notes

In [5]:
cos_notes = np.load('cos_notes.npy')
# cos_accords = np.load('cos_accords.npy')

---------
# Validation!
## Get similarity score comparisons for test and random

In [ ]:
def validation_dict(dataframe):
    """
    input: dataframe, perfumes dataframe
    returns: dict, keys are indices of perfumes, values are indices 
             of "reminds me of" perfumes that are contained in a list
    """
    # make new dict
    validation_dict = {}

    # for each row
    for df_ind in df[df.reminds != '[]'].index:
        # for each reminds
        for small_image in df.ix[df_ind].clean_reminds:
            # get index
            try:
                remind_index = df[df.image==('https://fimgs.net/images/perfume/nd.' + small_image[35:])].index[0]
            except IndexError:
                continue
                
            # make dict
            try:
                validation_dict[df_ind] += [remind_index]
            except KeyError:
                validation_dict[df_ind] = [remind_index]
    return validation_dict

In [ ]:
validation_dict = validation_dict(df)

In [ ]:
def get_similarity_score_comparisons(validation_dict, cos_matrix):
    """
    input:
        validation_dict: dict, keys are indices of perfumes, values are indices 
                         of "reminds me of" perfumes that are contained in a list
        cos_matrix: cosine similarity matrix, numpy
    returns: get similarity scores for test set and random set; two lists
    """
    test_sim_score = []
    random_sim_score = []
    
    for key in validation_dict.keys():
        for item in validation_dict[key]:
            test_sim_score.append(cos_matrix[key][item])
    print ('test mean and std:', np.mean(test_sim_score), np.std(test_sim_score))

    for i in range(len(test_sim_score)):
        random_sim_score.append(cos_matrix[np.random.randint(len(cos_matrix))][np.random.randint(len(cos_matrix))])
    print ('random mean and std:', np.mean(random_sim_score), np.std(random_sim_score))
    
    return test_sim_score, random_sim_score

In [ ]:
# cosine similarities for test and random "notes"
test_sim_notes, random_sim_notes = get_similarity_score_comparisons(validation_dict, cos_notes)

In [ ]:
# cosine similarities for test and random "accords"
test_sim_accords, random_sim_accords = get_similarity_score_comparisons(validation_dict, cos_accords)

In [ ]:
def plot_graphs(test_sim, random_sim):
    """
    input: test_sim and random_sim, two lists
    returns: None
    """
    # plot scatterplots for test_sim and random_sim
    pyplot.figure()
    pyplot.plot(test_sim, marker='.', linewidth=0, alpha=0.2, color='salmon')
    pyplot.plot(random_sim, marker='.', linewidth=0, alpha=0.2, color='cornflowerblue')
    pyplot.ylim([0,1])
    pyplot.ylabel('cosine-similarity score')
    pyplot.xlim([0,len(test_sim)])
    pyplot.xlabel('perfume')
    
    # plot similarity score curves
    pyplot.figure()
    pyplot.plot(sorted(test_sim), 'salmon')
    pyplot.plot(sorted(random_sim), 'cornflowerblue')
    pyplot.ylim([0,1])
    pyplot.xlim([0, len(random_sim)])
    
    # plot similarity score cumsum curves
    pyplot.figure()
    pyplot.plot(np.cumsum(sorted(test_sim, reverse=True)), 'salmon')
    pyplot.plot(np.cumsum(sorted(random_sim, reverse=True)), 'cornflowerblue')
    pyplot.xlim([0, len(random_sim)])
    
    # plot histograms
    pyplot.figure()
    sns.distplot(test_sim, bins=50, color='salmon', kde=False, hist=True, kde_kws={"shade": True})
    sns.distplot(random_sim, bins=50, color='cornflowerblue', kde=False, hist=True, kde_kws={"shade": True})
    sns.set_style('white')
    pyplot.xlim([0,1])
    pyplot.xlabel('cosine similarity score')
    pyplot.ylabel('count')

In [ ]:
notes_plot = plot_graphs(test_sim_notes, random_sim_notes)
pyplot.savefig('notes_plot.pdf')

In [ ]:
plot_graphs(test_sim_accords, random_sim_accords)

# Make permutation test

In [ ]:
def get_percentile(values_compiled):
    """
    input: values_compiled, where [0] is the p values of the true population and [1:] is the p values of the "fake" populations
    returns: percentiles of true population values
    """
    from scipy import stats

    values = [i[0] for i in values_compiled]

    return stats.percentileofscore(values, values[0], kind='rank'), values

In [ ]:
def permutation_test(test_set, random_set, number_of_loops):
    """
    input: test_set and random_set; two lists
           number_of_loops: int, how many iterations should permutation test loop through
    returns: permutation test values (percentile ranking and value) using two sample t-test as test statistic
    """
    from scipy.stats import ttest_ind
    true_stat = ttest_ind(test_set, random_set, equal_var=False)

    # merge test and random sets
    mixed = test_set + random_set
    
    fake_stats = []
    for i in range(number_of_loops):
        # make indices for the two diferent populations
        pop_1 = mixed[:]
        pop_1_index = np.random.choice(len(mixed), len(mixed)//2, replace=False)
        
        # make pop_2
        pop_2 = []
        for number in pop_1_index:
            pop_2.append(pop_1[number])

        # make pop_1
        for j in sorted(pop_1_index.tolist(), reverse=True):
            del (pop_1[j])

        fake_stats.append(ttest_ind(pop_1, pop_2, equal_var=False))
        
    fake_stats.insert(0, true_stat)
    
    pct, val = get_percentile(fake_stats)
    return pct, val

In [ ]:
percentile_acc, values_acc = permutation_test(test_sim_accords, 
                                      random_sim_accords, 100000)

In [ ]:
percentile_not, values_not = permutation_test(test_sim_notes, 
                                          random_sim_notes, 100000)

In [ ]:
print (percentile_acc)
print(percentile_not)

-----
# Get recommendations and ranking

## Get Bayesian rating

In [ ]:
df['rating_mean'] = df.rating.mean()
df['rating_ct_mean'] = df.rating_count.mean()

# get bayesian rating score for each perfume
df['bayesian_rating'] = ((df.rating_ct_mean*df.rating_mean) + (df.rating_count * df.rating_count * df.rating)) / (df.rating_ct_mean + df.rating_count)

## Get recommendation

In [9]:
def get_ebay_link(perfume_string):
    """
    input: perfume_string, string
    returns: link to ebay page, string
    """
    import ebaysdk
    from ebaysdk.finding import Connection as finding
    
    api = finding(siteid='EBAY-US', 
                  appid='CandiceY-Bettersc-PRD-b8dfd86bc-41f85e11',
                  config_file=None)

    api.execute('findItemsAdvanced', {
        'keywords': perfume_string,
        'categoryId' : ['180345'],
        'paginationInput': {
            'entriesPerPage': '25',
            'pageNumber': '1' 
        },
        'sortOrder': 'CurrentPriceHighest'
    })

    dictstr = api.response.dict()
    return dictstr['itemSearchURL']

In [17]:
def get_recommendations_df(perfume_string, df, cos_matrix, evaluator):
    """
    inputs:
        perfume_string: str, input perfume to get recommendation for
        df: perfume database
        cos_df: dataframe; cosine similarity matrix
    returns: dataframe, ranked by highest recommendation
    """
    if perfume_string in list(df.name):
        # get index of perfume string
        perfume_string_index = df[df.name == perfume_string].index
        
        # get cos-similarity values for that perfume
        similarity_vals = cos_matrix[perfume_string_index]
        
        # get indices for cos-similarity values
        ranked_recs = np.argsort(similarity_vals)[0][:-22:-1]
        
        # get dataframe of top 20 most similar perfumes
        df_to_return = df.iloc[ranked_recs]
        df_to_return = df_to_return.drop(perfume_string_index)
        
        top_hit = df_to_return.sort_values(by='bayesian_rating', ascending=False).head(1)
        rec_name = top_hit.name.tolist()[0]
        
        rec_why = top_hit[evaluator].tolist()[0][2:-2].split("), ('")
        rec_why = rec_why[0].split("', '")

        
        orig_why = df.ix[perfume_string_index][evaluator].tolist()[0][2:-2].split("), ('")
        orig_why = orig_why[0].split("', '")

        image = top_hit.image
                
        final_why = []

        for i in orig_why:
            if i in rec_why:
                final_why.append(i)

        if len(final_why) == 1:
            statement = 'We recommend this fragrance because it shares ' + str(final_why[0]).lower() + ' notes with ' + str(perfume_string) + '.'
        else:
            new_s = ""
            for i in final_why[:-1]:
                new_s += (i + ', ')
            statement = 'We recommend this fragrance because it shares ' + new_s.lower() + ' and ' + str(final_why[-1]).lower() + ' notes with ' + str(perfume_string) + '.'

        ebay = get_ebay_link(rec_name)
        
        rating = round(top_hit.rating.tolist()[0], 2)
        
        rating_count = int(top_hit.rating_count.tolist()[0])
        
        return rec_name, statement, ebay, image.iloc[0], rating, rating_count
    
    else:
        rec_name = "Sorry, we don't have enough data about this fragrance to construct a recommendation."
        statement = None
        ebay = None
        image = None
        rating = None
        rating_count = None
        return rec_name, statement, ebay, image, rating, rating_count

## Get recommendation - notes

In [18]:
get_recommendations_df('Cool Water Davidoff for men', df, cos_notes, 'notes')

('Burberry Men Burberry for men',
 'We recommend this fragrance because it shares mint, lavender, sandalwood, jasmine, geranium, musk, oakmoss, cedar,  and amber notes with Cool Water Davidoff for men.',
 'http://www.ebay.com/sch/180345/i.html?_nkw=Burberry+Men+Burberry+for+men&_ddo=1&_ipg=25&_pgn=1&_sop=3',
 'https://fimgs.net/images/perfume/nd.817.jpg',
 0.79,
 920)

# Wordcloud implementation

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

STOPWORDS.update(['perfume', 'perfumes', 'note', 'notes', 'smell', 'scent', 'fragrance', 'one', 'bottle'])
word_string = df.reviews_dict[19838]
wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          width=4800,
                          height=2000
                         ).generate(word_string)


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# Get names to put into website for dropdown menu

In [ ]:
all_names = df.name.tolist()
a = []
for all_name in all_names:
    all_name = all_name.replace('"', "'")
    a.append(all_name)


all_names.replace